## загрузка данных 

In [ ]:
# Загрузка и превью данных aim-2025-orcs
from pathlib import Path
import json, os, subprocess, zipfile, shutil
import pandas as pd

comp = "aim-2025-orcs"
data_dir = Path("data") / comp
data_dir.mkdir(parents=True, exist_ok=True)

workspace_cfg = Path("kaggle.json")
home_cfg_dir = Path.home() / ".kaggle"
home_cfg = home_cfg_dir / "kaggle.json"
if workspace_cfg.exists():
    home_cfg_dir.mkdir(exist_ok=True)
    data = json.loads(workspace_cfg.read_text())
    home_cfg.write_text(json.dumps(data))
    try:
        os.chmod(home_cfg, 0o600)
    except Exception:
        pass
    os.environ["KAGGLE_USERNAME"] = data.get("username", "")
    os.environ["KAGGLE_KEY"] = data.get("key", "")
else:
    print("kaggle.json не найден рядом с ноутбуком — пропускаю установку cred")

zip_files = list(data_dir.glob("*.zip"))
extracted_exists = any(
    p.is_file() and p.suffix.lower() in {".csv", ".tsv", ".parquet", ".json", ".parq", ".pq"}
    for p in data_dir.iterdir()
)

def try_cli_download() -> bool:
    if shutil.which("kaggle") is None:
        print("kaggle CLI not found in PATH")
        return False
    res = subprocess.run(
        ["kaggle", "competitions", "download", "-c", comp, "-p", str(data_dir)],
        capture_output=True,
        text=True,
    )
    if res.returncode != 0:
        print("kaggle CLI failed:")
        if res.stdout:
            print(res.stdout)
        if res.stderr:
            print(res.stderr)
        return False
    return True

def api_download():
    try:
        from kaggle import api
    except ImportError:
        raise RuntimeError("Install kaggle: pip install kaggle")
    api.authenticate()
    api.competition_download_files(comp, path=str(data_dir), quiet=False)

def ensure_data():
    if extracted_exists:
        print(f"Found extracted files in {data_dir.resolve()}, skip download.")
        return
    if zip_files:
        for zip_path in zip_files:
            with zipfile.ZipFile(zip_path) as zf:
                zf.extractall(data_dir)
        print(f"Used existing zip files in {data_dir.resolve()} and extracted.")
        return
    used_cli = try_cli_download()
    if not used_cli:
        api_download()
    for zip_path in data_dir.glob("*.zip"):
        with zipfile.ZipFile(zip_path) as zf:
            zf.extractall(data_dir)
    print(f"Downloaded and extracted to {data_dir.resolve()}")

ensure_data()

files = sorted([*data_dir.glob("*.parquet"), *data_dir.glob("*.csv")])
if not files:
    available = sorted(p.name for p in data_dir.glob("*"))
    print(f"Файлы не найдены в {data_dir.resolve()}. Доступно: {available}")
else:
    def preview(path):
        print(f"--- {path.name} ---")
        if path.suffix.lower() == ".parquet":
            display(pd.read_parquet(path).head(3))
        else:
            display(pd.read_csv(path, nrows=3))

    targets = {"orcs": None, "empl": None}
    for p in files:
        name = p.name.lower()
        if "orc" in name and targets["orcs"] is None:
            targets["orcs"] = p
        if "empl" in name and targets["empl"] is None:
            targets["empl"] = p

    for key, path in targets.items():
        if path is None:
            print(f"Файл с '{key}' в имени не найден. Доступно: {[p.name for p in files]}")
            continue
        preview(path)



### Алгоритм нормализации имён
- все строки в нижний регистр, `ё→е`, `й→и`
- явный мусор (`none`, `нет`, `нету`, `отсутствует`, `ноне`, пустые) очищается в `null`
- дефис заменяется пробелом, части двойного имени обрабатываются отдельно
- визуальные латинские омоглифы транслитерируются в кириллицу (`a→а`, `p→р`, `y→у`, `o→о`, `c→с`, `x→х`, `e→е` и т.д.)
- всё остальное вне русского алфавита (кроме пробела) превращается в джокер `*`
- если в строке нет букв или букв меньше, чем спецсимволов (`*`), значение очищается
- итог: только кириллические буквы + пробелы + `*`, лишние пробелы схлопываются


In [4]:
import json
import pandas as pd
from pathlib import Path

classes = ["corrected", "untouched", "ambiguous", "empty", "no_match"]

report_dirs = {
    "employees": Path("reports/name/employees"),
    "orcs": Path("reports/name/orcs"),
}

# Частоты по каждому датасету
for label, report_dir in report_dirs.items():
    dist_path = report_dir / "class_distribution.json"
    if dist_path.exists():
        counts = json.loads(dist_path.read_text(encoding="utf-8"))
        print(f"\n=== class counts: {label} ===")
        display(pd.Series(counts, name="count"))
    else:
        print(f"Нет файла {dist_path}")

# Примеры: две таблицы рядом (employees vs orcs) для каждого класса
for cls in classes:
    emp_samples_path = report_dirs["employees"] / "class_samples.json"
    orc_samples_path = report_dirs["orcs"] / "class_samples.json"
    if not emp_samples_path.exists() or not orc_samples_path.exists():
        print(f"Пропущен класс {cls}: нет файлов samples")
        continue
    emp_samples = json.loads(emp_samples_path.read_text(encoding="utf-8")).get(cls, [])
    orc_samples = json.loads(orc_samples_path.read_text(encoding="utf-8")).get(cls, [])

    emp_df = pd.DataFrame(emp_samples).head(20).add_suffix("_employees")
    orc_df = pd.DataFrame(orc_samples).head(20).add_suffix("_orcs")
    side = pd.concat([
        emp_df.reset_index(drop=True),
        orc_df.reset_index(drop=True)
    ], axis=1)
    print(f"\nКласс: {cls} (employees vs orcs)")
    display(side)



=== class counts: employees ===


ambiguous    143349
corrected    294718
empty            84
no_match     821101
untouched    752507
Name: count, dtype: int64


=== class counts: orcs ===


ambiguous     1463
corrected     4692
empty            4
no_match     36576
untouched     4898
Name: count, dtype: int64


Класс: corrected (employees vs orcs)


...

,normalized_name_employees,name_corrected_employees,is_name_employees,normalized_name_orcs,name_corrected_orcs,is_name_orcs
0,евдокия,евдокия,False,иян,иян,False
1,александра,александра,False,раиса,раиса,False
2,клавдия,клавдия,False,асмат,асмат,False
3,александра,александра,False,валентина,валентина,False
4,олеся,олеся,False,марина,марина,False
5,маря,маря,False,раиса,раиса,False
6,валентина,валентина,False,ома,ома,False
7,ивна,ивна,False,демир,демир,False
8,антонина,антонина,False,валентина,валентина,False
9,раиса,раиса,False,фаиса,фаиса,False



Класс: empty (employees vs orcs)


,normalized_name_employees,name_corrected_employees,is_name_employees,normalized_name_orcs,name_corrected_orcs,is_name_orcs
0,None,,False,None,,False
1,None,,False,None,,False
2,None,,False,None,,False
3,None,,False,None,,False
4,None,,False,NaN,NaN,NaN
5,None,,False,NaN,NaN,NaN
6,None,,False,NaN,NaN,NaN
7,None,,False,NaN,NaN,NaN
8,None,,False,NaN,NaN,NaN
9,None,,False,NaN,NaN,NaN



Класс: no_match (employees vs orcs)


,normalized_name_employees,name_corrected_employees,is_name_employees,normalized_name_orcs,name_corrected_orcs,is_name_orcs
0,ирина,ирина,False,кувцри,кувцри,False
1,еснеия,еснеия,False,нурбеи,нурбеи,False
2,оскана,оскана,False,курбоншо,курбоншо,False
3,тамара,тамара,False,ди*бази,ди*бази,False
4,ол*ся,ол*ся,False,бриуто,бриуто,False
5,ия*ла,ия*ла,False,людмила,людмила,False
6,лидия,лидия,False,ханис,ханис,False
7,любовь,любовь,False,цезарии,цезарии,False
8,люмила,люмила,False,кубоныч,кубоныч,False
9,дильмурот,дильмурот,False,пот*,пот*,False


In [1]:
# Отчёт по фамилиям: классы и 20 примеров по каждому классу (employees vs orcs)
import json
import pandas as pd
from pathlib import Path

classes = ["corrected", "untouched", "ambiguous", "empty", "no_match"]
report_dirs = {
    "employees": Path("reports/surname/employees"),
    "orcs": Path("reports/surname/orcs"),
}

for label, report_dir in report_dirs.items():
    dist_path = report_dir / "class_distribution.json"
    if dist_path.exists():
        counts = json.loads(dist_path.read_text(encoding="utf-8"))
        print(f"\n=== surname class counts: {label} ===")
        display(pd.Series(counts, name="count"))
    else:
        print(f"Нет файла {dist_path}")

for cls in classes:
    emp_path = report_dirs["employees"] / "class_samples.json"
    orc_path = report_dirs["orcs"] / "class_samples.json"
    if not emp_path.exists() or not orc_path.exists():
        print(f"Пропущен класс {cls}: нет файлов samples")
        continue
    emp_samples = json.loads(emp_path.read_text(encoding="utf-8")).get(cls, [])
    orc_samples = json.loads(orc_path.read_text(encoding="utf-8")).get(cls, [])

    emp_df = pd.DataFrame(emp_samples).head(20).add_suffix("_employees")
    orc_df = pd.DataFrame(orc_samples).head(20).add_suffix("_orcs")
    side = pd.concat([
        emp_df.reset_index(drop=True),
        orc_df.reset_index(drop=True)
    ], axis=1)
    print(f"\nКласс фамилии: {cls} (employees vs orcs)")
    display(side)




=== surname class counts: employees ===


ambiguous    418095
corrected    429428
empty            37
no_match     418458
untouched    745741
Name: count, dtype: int64


=== surname class counts: orcs ===


ambiguous     9825
corrected     9992
no_match     19428
untouched     8388
Name: count, dtype: int64


Класс фамилии: corrected (employees vs orcs)


...

,normalized_surname_employees,surname_corrected_employees,is_surname_employees,normalized_surname_orcs,surname_corrected_orcs,is_surname_orcs
0,огров,огров,False,гулаидин,гулаидин,False
1,броин,броин,False,суая,суая,False
2,бурово,бурово,False,зрадовскии,зрадовскии,False
3,ерченков,ерченков,False,естаев,естаев,False
4,бухтино,бухтино,False,холунов,холунов,False
5,улюк,улюк,False,белва,белва,False
6,маст*ков,маст*ков,False,мерзаев,мерзаев,False
7,хлопенова,хлопенова,False,коцыло,коцыло,False
8,журино,журино,False,рузив,рузив,False
9,смирново,смирново,False,шеффлер,шеффлер,False



Класс фамилии: empty (employees vs orcs)


,normalized_surname_employees,surname_corrected_employees,is_surname_employees
0,None,,False
1,None,,False
2,None,,False
3,None,,False
4,None,,False
5,None,,False
6,None,,False
7,None,,False
8,None,,False
9,None,,False



Класс фамилии: no_match (employees vs orcs)


,normalized_surname_employees,surname_corrected_employees,is_surname_employees,normalized_surname_orcs,surname_corrected_orcs,is_surname_orcs
0,чашкне,чашкне,False,венженовскии,венженовскии,False
1,корпало,корпало,False,борбуцк*я,борбуцк*я,False
2,скердо,скердо,False,чернпяшуев,чернпяшуев,False
3,каибх*нов,каибх*нов,False,левурддк,левурддк,False
4,торыхчиев,торыхчиев,False,филашихни,филашихни,False
5,тиллекв,тиллекв,False,боатырив,боатырив,False
6,авакиви,авакиви,False,жекал,жекал,False
7,па*дуков,па*дуков,False,чоговадзе,чоговадзе,False
8,могаево,могаево,False,цаиек,цаиек,False
9,якиш*в*,якиш*в*,False,приблудово,приблудово,False


In [4]:
# Отчёт по отчествам: классы и 20 примеров (employees vs orcs)
import json
import pandas as pd
from pathlib import Path

classes = ["corrected", "untouched", "ambiguous", "empty", "no_match"]
report_dirs = {
    "employees": Path("reports/patronymic/employees"),
    "orcs": Path("reports/patronymic/orcs"),
}

for label, report_dir in report_dirs.items():
    dist_path = report_dir / "class_distribution.json"
    if dist_path.exists():
        counts = json.loads(dist_path.read_text(encoding="utf-8"))
        print(f"\n=== patronymic class counts: {label} ===")
        display(pd.Series(counts, name="count"))
    else:
        print(f"Нет файла {dist_path}")

for cls in classes:
    emp_path = report_dirs["employees"] / "class_samples.json"
    orc_path = report_dirs["orcs"] / "class_samples.json"
    if not emp_path.exists() or not orc_path.exists():
        print(f"Пропущен класс {cls}: нет файлов samples")
        continue
    emp_samples = json.loads(emp_path.read_text(encoding="utf-8")).get(cls, [])
    orc_samples = json.loads(orc_path.read_text(encoding="utf-8")).get(cls, [])

    emp_df = pd.DataFrame(emp_samples).head(20).add_suffix("_employees")
    orc_df = pd.DataFrame(orc_samples).head(20).add_suffix("_orcs")
    side = pd.concat([
        emp_df.reset_index(drop=True),
        orc_df.reset_index(drop=True)
    ], axis=1)
    print(f"\nКласс отчества: {cls} (employees vs orcs)")
    display(side)




=== patronymic class counts: employees ===


ambiguous     64985
corrected    737450
empty          5179
no_match     602557
untouched    601588
Name: count, dtype: int64


=== patronymic class counts: orcs ===


ambiguous      827
corrected     9579
empty           59
no_match     33778
untouched     3390
Name: count, dtype: int64


Класс отчества: corrected (employees vs orcs)


...

,normalized_patronymic_employees,patronymic_corrected_employees,is_patronymic_employees,normalized_patronymic_orcs,patronymic_corrected_orcs,is_patronymic_orcs
0,анатольеви,анатоль*еви,False,влодимировна,влодимировна,False
1,васильевич,васильевич,False,егоревич,егоревич,False
2,васильевна,васильевна,False,осифович,осифович,False
3,васильевич,васильевич,False,лександровна,лександровна,False
4,васильевна,васильевна,False,зелемови,зелем*ови,False
5,якобови,якоб*ови,False,темболатови,темболат*ови,False
6,виколаевна,виколаевна,False,васильевна,васильевна,False
7,васильевна,васильевна,False,васильеви,василь*еви,False
8,ванович,ванович,False,роиановна,роиановна,False
9,влодимирович,влодимирович,False,а*аевич,а*аевич,False



Класс отчества: empty (employees vs orcs)


,normalized_patronymic_employees,patronymic_corrected_employees,is_patronymic_employees,normalized_patronymic_orcs,patronymic_corrected_orcs,is_patronymic_orcs
0,None,,False,None,,False
1,None,,False,None,,False
2,None,,False,None,,False
3,None,,False,None,,False
4,None,,False,None,,False
5,None,,False,None,,False
6,None,,False,None,,False
7,None,,False,None,,False
8,None,,False,None,,False
9,None,,False,None,,False



Класс отчества: no_match (employees vs orcs)


,normalized_patronymic_employees,patronymic_corrected_employees,is_patronymic_employees,normalized_patronymic_orcs,patronymic_corrected_orcs,is_patronymic_orcs
0,омриовна,омриовна,False,мохаммда як*б,мохаммда як*б,False
1,садмпровис,садмпровис,False,саханович,саханович,False
2,изанощнэ,изанощнэ,False,затеовна,затеовна,False
3,арамашовна,арамашовна,False,ондревна,ондревна,False
4,иъатоврл,иъатоврл,False,аломудинович,аломудинович,False
5,гасан агаевеч,гасан агаевеч,False,словоммр,словоммр,False
6,влаимиаовна,влаимиаовна,False,чумобеко*ич,чумобеко*ич,False
7,бобомуродовеч,бобомуродовеч,False,гендрикоич,гендрикоич,False
8,восил*евич,восил*евич,False,м*каилович,м*каилович,False
9,мутагаровна,мутагаровна,False,к*чичиевич,к*чичиевич,False
